函数就像机械加工，着重于处理过程；类则关注数据本身，使其“活”过来。  
类存在两种关系：继承(inhertiance)自某个族类，组合(composition)了哪些部件。   
建议将类与业务逻辑分离。逻辑代表业务处理过程，其通常被设计为无状态，用上下文传递数据。业务逻辑讲究效率和并发性，以及相对固定的执行过程。逻辑适合以函数分段实现，按特定顺序组装，并以模块存储。这些特点都不适合以数据为核心，且注重家族遗传，用多实例体现个性的类来实现。

In [1]:
def test():
    class X:
        data =100
        def get(self):
            return self.data


In [2]:
import dis
dis.dis(test)

  2           0 LOAD_BUILD_CLASS
              2 LOAD_CONST               1 (<code object X at 0x111a47540, file "<ipython-input-1-47c898501eae>", line 2>)
              4 LOAD_CONST               2 ('X')
              6 MAKE_FUNCTION            0
              8 LOAD_CONST               2 ('X')
             10 CALL_FUNCTION            2
             12 STORE_FAST               0 (X)
             14 LOAD_CONST               0 (None)
             16 RETURN_VALUE


先创建了X函数，其内容是属性设置和方法创建。随后该函数被当作参数传递给buildins.\_\_build_class__调用，这里其实就是元类在执行。

In [4]:
dis.dis(test.__code__.co_consts[1])

  2           0 LOAD_NAME                0 (__name__)
              2 STORE_NAME               1 (__module__)
              4 LOAD_CONST               0 ('test.<locals>.X')
              6 STORE_NAME               2 (__qualname__)

  3           8 LOAD_CONST               1 (100)
             10 STORE_NAME               3 (data)

  4          12 LOAD_CONST               2 (<code object get at 0x111a47c90, file "<ipython-input-1-47c898501eae>", line 4>)
             14 LOAD_CONST               3 ('test.<locals>.X.get')
             16 MAKE_FUNCTION            0
             18 STORE_NAME               4 (get)
             20 LOAD_CONST               4 (None)
             22 RETURN_VALUE


- 类型有自己的名字空间，存储当前类定义的属性和方法，这其中并不包括所继承的祖先成员，其以引用关联祖先类型，无须复制本地
- 实例会存储所有继承层次的实例字段，这些都属于其私有数据
- 类型的名字空间返回mappingproxy只读视图，不允许直接修改；实例的名字空间就是普通字典，可直接修改
- dir函数搜索所有可访问成员的名字，vars直接返回\_\_dict__属性



In [5]:
def enclosing():
    a = 'enclosing.a'
    class A:
        a = 'A.a'
        def test(self):
            print('E.a=',a)
        print('A.a=',a)
        print('A.locals=',locals())
    A().test()
enclosing()

A.a= A.a
A.locals= {'__module__': '__main__', '__qualname__': 'enclosing.<locals>.A', 'a': 'A.a', 'test': <function enclosing.<locals>.A.test at 0x111a4fd90>}
E.a= enclosing.a


双下划线开头的私有字段会被重命名，这也导致其继承类型无法访问重命名后的基类成员，这也是单下划线私有字段的一个原因

- 属性（property）机制就是将读、写和删除操作映射到指定的方法调用上，从而实现操作控制
- 属性可实现延迟初始化，无须提前准备数据。且属性方法调用未必会绑定字段，可按需要从任意数据源获取
- 尽管属性保存于类型的名字空间，但其优先级高于同名实例字段

In [8]:
class X:
    def __init__(self, name):
        self.__name = name
    @property
    def name(self):
        return self.__name
    @name.setter
    def name(self, value):
        self.__name = value

In [9]:
vars(X)

mappingproxy({'__dict__': <attribute '__dict__' of 'X' objects>,
              '__doc__': None,
              '__init__': <function __main__.X.__init__>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'X' objects>,
              'name': <property at 0x111a7a188>})

In [11]:
vars(X('haha'))

{'_X__name': 'haha'}